In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("car_price_prediction.csv")
df

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,45798355,8467,-,MERCEDES-BENZ,CLK 200,1999,Coupe,Yes,CNG,2.0 Turbo,300000 km,4.0,Manual,Rear,02-Mar,Left wheel,Silver,5
19233,45778856,15681,831,HYUNDAI,Sonata,2011,Sedan,Yes,Petrol,2.4,161600 km,4.0,Tiptronic,Front,04-May,Left wheel,Red,8
19234,45804997,26108,836,HYUNDAI,Tucson,2010,Jeep,Yes,Diesel,2,116365 km,4.0,Automatic,Front,04-May,Left wheel,Grey,4
19235,45793526,5331,1288,CHEVROLET,Captiva,2007,Jeep,Yes,Diesel,2,51258 km,4.0,Automatic,Front,04-May,Left wheel,Black,4


In [4]:
df.drop(labels=["ID","Doors"],axis=1,inplace=True)

In [5]:
df.isna().sum()

Price               0
Levy                0
Manufacturer        0
Model               0
Prod. year          0
Category            0
Leather interior    0
Fuel type           0
Engine volume       0
Mileage             0
Cylinders           0
Gear box type       0
Drive wheels        0
Wheel               0
Color               0
Airbags             0
dtype: int64

In [6]:

df["Levy"]=df["Levy"].replace("-",np.nan)

In [7]:
df.isna().sum()

Price                  0
Levy                5819
Manufacturer           0
Model                  0
Prod. year             0
Category               0
Leather interior       0
Fuel type              0
Engine volume          0
Mileage                0
Cylinders              0
Gear box type          0
Drive wheels           0
Wheel                  0
Color                  0
Airbags                0
dtype: int64

In [15]:
x = df.drop(labels="Price",axis=1)
x
y = df["Price"]
y

0        13328
1        16621
2         8467
3         3607
4        11726
         ...  
19232     8467
19233    15681
19234    26108
19235     5331
19236      470
Name: Price, Length: 19237, dtype: int64

In [16]:
from sklearn.preprocessing import OneHotEncoder
char_cols = df.select_dtypes(include='object').columns
char_cols
num_cols = df.select_dtypes(include=np.number).columns
num_cols

Index(['Price', 'Prod. year', 'Cylinders', 'Airbags'], dtype='object')

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [20]:
from sklearn.impute import SimpleImputer

impute = SimpleImputer(strategy="median")

# Fit using training column (2D)
impute.fit(x_train[["Levy"]])

# Transform both sets correctly
x_train["Levy"] = impute.transform(x_train[["Levy"]])
x_test["Levy"] = impute.transform(x_test[["Levy"]])


In [21]:

from sklearn.preprocessing import OneHotEncoder
# 2. Create encoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
# 3. Fit ONLY on training data
encoder.fit(x_train[char_cols])
# 4. Transform both sets using same encoder
X_train_encoded = encoder.transform(x_train[char_cols])
X_test_encoded  = encoder.transform(x_test[char_cols])

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(X_train_encoded)
x_test = scaler.transform(X_test_encoded)

In [23]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train,y_train)


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [24]:
y_pred = lr.predict(x_test)

In [27]:
from sklearn.metrics import mean_absolute_percentage_error,r2_score
print(mean_absolute_percentage_error(y_test,y_pred))
print(r2_score(y_pred,y_test))

61.51302540492528
0.017524770788760358


In [31]:
from sklearn.preprocessing import PolynomialFeatures
fetures = PolynomialFeatures(degree=2)
x_poly = fetures.fit_transform(x_train)
model1 = LinearRegression()
model1.fit(x_poly,y)
# poly_pred = model1.predict(x_poly)

MemoryError: Unable to allocate 4.17 TiB for an array with shape (15389, 37234135) and data type float64